In [2]:
import os
from PIL import Image
from ultralytics import YOLO
from PIL import Image, ImageDraw, ImageFont
import random
import numpy as np

/home/mees/.local/lib/python3.8/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (4.0.0)/charset_normalizer (3.0.1) doesn't match a supported version!
  warnings.warn(


In [3]:
def generate_lp_number():
    # Set up the allowed characters
    allowed_letters = 'BCDFGHJKLMNOPRSTVWXYZ'
    allowed_numbers = '0123456789'

    # Define the cases
    cases = [
        # Case 1: 99-XX-XX
        random.choice(allowed_numbers) + random.choice(allowed_numbers) + '-' + random.choice(allowed_letters) + random.choice(allowed_letters) + '-' + random.choice(allowed_letters) + random.choice(allowed_letters),

        # Case 2: 99-XXX-9
        random.choice(allowed_numbers) + random.choice(allowed_numbers) + '-' + random.choice(allowed_letters) + random.choice(allowed_letters) + random.choice(allowed_letters) + '-' + random.choice(allowed_numbers),

        # Case 3: 9-XXX-99
        random.choice(allowed_numbers) + '-' + random.choice(allowed_letters) + random.choice(allowed_letters) + random.choice(allowed_letters) + '-' + random.choice(allowed_numbers) + random.choice(allowed_numbers),

        # Case 4: XX-999-X
        random.choice(allowed_letters) + random.choice(allowed_letters) + '-' + random.choice(allowed_numbers) + random.choice(allowed_numbers) + random.choice(allowed_numbers) + '-' + random.choice(allowed_letters),

        # Case 5: X-999-XX
        random.choice(allowed_letters) + '-' + random.choice(allowed_numbers) + random.choice(allowed_numbers) + random.choice(allowed_numbers) + '-' + random.choice(allowed_letters) + random.choice(allowed_letters)
    ]

    # Return a random case
    return random.choice(cases)

In [4]:
def generate_lp_image():
    # Load the font
    font = ImageFont.truetype('../lp_template_files/font_dutch_lp.ttf', 220)
    lp_image = Image.open('../lp_template_files/template_dutch_lp.png')

    # Create a drawing context
    draw = ImageDraw.Draw(lp_image)

    # Generate a random license plate number
    license_plate = generate_lp_number()

    # Calculate the width and height of the text
    text_bbox = draw.textbbox((0, 0), license_plate, font)
    text_width = text_bbox[2] - text_bbox[0]
    text_height = text_bbox[3] - text_bbox[1]

    # Calculate the position of the text
    text_x = (lp_image.width - text_width) / 2 + 60
    text_y = (lp_image.height - text_height) / 2 - 45

    # Draw the text on the image
    draw.text((text_x, text_y), license_plate, fill='black', font=font)

    return lp_image


In [5]:
def rotate_image(image, angle):
    return image.rotate(angle, expand=True)

In [6]:
def transform_pixel_coordinates(pixel_coordinates, angle, image, rotated_image):
    x, y = pixel_coordinates

    center = (image.width / 2, image.height / 2)
    transformed_center = (rotated_image.width / 2, rotated_image.height / 2)

    angle_radians = -np.deg2rad(angle)

    x -= center[0]
    y -= center[1]

    x_transformed = x * np.cos(angle_radians) - y * np.sin(angle_radians)
    y_transformed = x * np.sin(angle_radians) + y * np.cos(angle_radians)

    x_transformed += transformed_center[0]
    y_transformed += transformed_center[1]

    return int(x_transformed), int(y_transformed)

In [7]:
def rotate_image_and_transform_coordinates(image, angle, pixel_coordinates):
    rotated_image = rotate_image(image, angle)
    transformed_pixel_coordinates = transform_pixel_coordinates(
        pixel_coordinates, angle, image, rotated_image
    )
    return rotated_image, transformed_pixel_coordinates

In [88]:
def create_dataset(raw_image_path, amount, size):

    image_files = os.listdir(raw_image_path)

    for i in range(amount):

        # load random image and resize
        img = Image.open(os.path.join(raw_image_path, image_files[i]))
        img = img.resize((size, size))

        # generate lp image
        lp_img = generate_lp_image()

        width_before_rot, height_before_rot = lp_img.size

        # rotate lp with random angle
        angle = np.random.uniform(-30,30)
        rotated_lp_img, top_left_coordinate = rotate_image_and_transform_coordinates(lp_img, angle, (0,0))
        _, top_right_coordinate = rotate_image_and_transform_coordinates(lp_img, angle, (width_before_rot, 0))
        _, bottom_left_coordinate = rotate_image_and_transform_coordinates(lp_img, angle, (0, height_before_rot))
        _, bottom_right_coordinate = rotate_image_and_transform_coordinates(lp_img, angle, (width_before_rot, height_before_rot))
    
        width, height = rotated_lp_img.size
        # resize lp image
        resize_param = np.random.uniform(5, 30)

        # resize the width and height before rotation
        width_before_rot, height_before_rot = width_before_rot/resize_param, height_before_rot/resize_param


        rotated_lp_img = rotated_lp_img.resize((int(width/resize_param), int(height/resize_param)))

        # paste coordinates
        x, y = np.random.randint(0, size - rotated_lp_img.width), np.random.randint(0, size - rotated_lp_img.height)

        # resize the coordinates before rotation
        top_left_coordinate = (top_left_coordinate[0]/resize_param, top_left_coordinate[1]/resize_param)
        top_left_coordinate = (top_left_coordinate[0] + x, top_left_coordinate[1] + y)

        top_right_coordinate = (top_right_coordinate[0]/resize_param, top_right_coordinate[1]/resize_param)
        top_right_coordinate = (top_right_coordinate[0] + x, top_right_coordinate[1] + y)

        bottom_left_coordinate = (bottom_left_coordinate[0]/resize_param, bottom_left_coordinate[1]/resize_param)
        bottom_left_coordinate = (bottom_left_coordinate[0] + x, bottom_left_coordinate[1] + y)

        bottom_right_coordinate = (bottom_right_coordinate[0]/resize_param, bottom_right_coordinate[1]/resize_param)
        bottom_right_coordinate = (bottom_right_coordinate[0] + x, bottom_right_coordinate[1] + y)

        # paste rotated lp ontop of random image
        img.paste(rotated_lp_img, (x,y), rotated_lp_img)

        # save image
        img.save(f'../../images/first_test/images/train/{i}.png')

        # save labels
        with open(f'../../images/first_test/labels/train/{i}.txt', 'w') as f:
            f.write(f'0 {top_left_coordinate[0]/size} {top_left_coordinate[1]/size} {top_right_coordinate[0]/size} {top_right_coordinate[1]/size} {bottom_right_coordinate[0]/size} {bottom_right_coordinate[1]/size} {bottom_left_coordinate[0]/size} {bottom_left_coordinate[1]/size}')

create_dataset('../../images/raw_random', 1000, 640)


OSError: image file is truncated (89 bytes not processed)